# Advertisment

## Easy as MATLAB, flexible as Python, deep as Lisp and fast as Fortran
John F. Gibson, [*WhyJulia*](https://github.com/johnfgibson/whyjulia/blob/master/1-whyjulia.ipynb).

## 1. Some mathematical code

2×2 Array{Complex{Int64},2}:
 0+1im  0+0im
 0+0im  0-1im

2×2 Array{Int64,2}:
 1  0
 0  1

1000

In [5]:
n * σ₁

2×2 Array{Int64,2}:
    0  1000
 1000     0

In [6]:
σ₂ * σ₁

2×2 Array{Complex{Int64},2}:
 0-1im  0+0im
 0+0im  0+1im

In [7]:
x = 1
y = 2

2

In [8]:
println("$x + $y = $(x + y)")

1 + 2 = 3


In [9]:
println("σ₂ = $(σ₂)")

σ₂ = Complex{Int64}[0+0im 0-1im; 0+1im 0+0im]


In [ ]:
1//3

In [ ]:
1//3 + 1//2

In [ ]:
1/3

In [ ]:
convert(Rational{Int32}, 1/3)

In [ ]:
convert(Rational{Int64}, 1/3)

In [ ]:
a = "Zakład Teorii "
b = "Pola"

In [ ]:
a * b

In [ ]:
"pole, "^7

In [ ]:
v₂ = [0 -2 3]

In [ ]:
?eigen

In [ ]:
eigen_A.vectors

# 2. Multiple dispatch

In [ ]:
f(x) = 2(x + 1)^2

In [ ]:
f(0)

In [ ]:
f(-1)

In [ ]:
f(1)

In [ ]:
f(σ₁)

In [ ]:
oneunit(1)

In [ ]:
oneunit(Int64)

In [ ]:
oneunit([1 2; 3 4])

In [ ]:
@code_typed f(0)

In [ ]:
@code_typed f(0.0)

In [ ]:
@code_warntype f(0)

In [ ]:
@code_lowered f(0)

In [ ]:
@code_native f(0)

In [ ]:
@code_llvm f(0)

In [ ]:
@code_typed g(0)

In [ ]:
@code_warntype g(0)

In [ ]:
@code_lowered g(0)

In [ ]:
@code_llvm g(0)

In [ ]:
@code_native g(0)

In [ ]:
@code_typed g(σ₁)

In [ ]:
@code_warntype g(σ₁)

In [ ]:
@code_lowered g(σ₁)

In [ ]:
@code_llvm g(σ₁)

In [ ]:
@code_native g(σ₁)

In [ ]:
methods(eigen)

# 3. Random numbers and matrics

In [ ]:
rand(5)

In [ ]:
randn(5)

In [ ]:
rand(4, 4)

In [ ]:
rand(4, 4, 4)

In [ ]:
1:4

In [ ]:
A = rand(1:4, 3, 3)

In [ ]:
using Colors

In [ ]:
distinguishable_colors(20)

In [ ]:
palette = distinguishable_colors(20)

In [ ]:
rand(palette, 5, 5)

# 4. There are lies, big lies and benchmarks

Examples taken from
[*Julia is fast*](https://github.com/JuliaComputing/JuliaBoxTutorials/blob/master/introductory-tutorials/intro-to-julia/09.%20Julia%20is%20fast.ipynb) at JuliaComputing GitHub account.

We test speed of Julia using simple case of function summing elements of vector $V$ with $N = 10^{ 7 }$ random elements
\begin{equation}
    \textrm{sum}(V) = \sum_{ i = 1 }^{ N } V_{ i }.
\end{equation}

## Function in Julia are faster that "free" code.

In [ ]:
using Plots, Libdl, BenchmarkTools, Statistics
gr()
V = rand(10^7)  # 1D vector of random numbers, uniform on [0,1)

In [ ]:
sum(V) / 10^7

## 4.1 The C language

I don't write this C code.

In [ ]:
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

Approx

In [ ]:
(c_sum(V) - sum(V)) / 10^7

In [ ]:
c_bench = @benchmark c_sum($V)

In [ ]:
d_min = Dict("C" => minimum(c_bench.times) / 1e6)  # We creat efficient dictionary
d_med = Dict("C" => median(c_bench.times) / 1e6)

In [ ]:
t = c_bench.times / 1e6

In [ ]:
min_t = 4
max_t = 10.5

In [ ]:
t = c_bench.times / 1e6  # times in milliseconds

histogram(t, bins=500,
             xlim=(min_t, max_t),
             xlabel="milliseconds", ylabel="count", label="")

## 4.2 C with -ffast-math

I don't write this code either.

In [ ]:
const Clib_fastmath = tempname()   # make a temporary file

# The same as above but with a -ffast-math flag added
open(`gcc -fPIC -O3 -msse3 -xc -shared -ffast-math -o $(Clib_fastmath * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum_fastmath(X::Array{Float64}) = ccall(("c_sum", Clib_fastmath), Float64, (Csize_t, Ptr{Float64}), length(X), X)

In [ ]:
c_fastmath_bench = @benchmark $c_sum_fastmath($V)
d_min["C -ffast-math"] = minimum(c_fastmath_bench.times) / 1e6  # in milliseconds
d_med["C -ffast-math"] = median(c_fastmath_bench.times) / 1e6

t = c_fastmath_bench.times / 1e6 # times in milliseconds

histogram(t, bins=500,
             xlim=(min_t, max_t),
             xlabel="milliseconds", ylabel="count", label="")

## 4.3 Julia (built-in)

In [ ]:
@which sum(V)  # This code on GitHub tell you probably nothing

In [ ]:
j_bench = @benchmark sum($V)
d_min["Julia built-in"] = minimum(j_bench.times) / 1e6  # in milliseconds
d_med["Julia built-in"] = median(j_bench.times) / 1e6

t = j_bench.times / 1e6 # times in milliseconds

histogram(t, bins=500,
             xlim=(min_t, max_t),
             xlabel="milliseconds", ylabel="count", label="")

## 4.4 Julia hand-written

In [ ]:
function my_sum(V)   
    s = 0.0 # s = zero(eltype(a))
    for v in V
        s += v
    end
    s
end

j_bench_hand = @benchmark my_sum($V)
d_min["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6  # in milliseconds
d_med["Julia hand-written"] = median(j_bench_hand.times) / 1e6

t = j_bench_hand.times / 1e6  # times in milliseconds

histogram(t, bins=500,
             xlim=(min_t, max_t),
             xlabel="milliseconds", ylabel="count", label="")

## 4.5 Julia hand-written with @simd

In [ ]:
?@simd

In [ ]:
function my_sum_simd(V)   
    s = 0.0  # s = zero(eltype(A))
    @simd for v in V
        s += v
    end
    s
end

j_bench_hand_simd = @benchmark my_sum_simd($V)
d_min["Julia hand-written simd"] = minimum(j_bench_hand_simd.times) / 1e6  # in milliseconds
d_med["Julia hand-written simd"] = median(j_bench_hand_simd.times) / 1e6

t = j_bench_hand_simd.times / 1e6 # times in milliseconds

histogram(t, bins=500,
             xlim=(min_t, max_t),
             xlabel="milliseconds", ylabel="count", label="")

### C with -ffast-math again

In [ ]:
t = c_fastmath_bench.times / 1e6  # times in milliseconds

histogram(t, bins=500,
             xlim=(min_t, max_t),
             xlabel="milliseconds", ylabel="count", label="")

## 4.6 Sumary

In [ ]:
println("Minimal time of computing sum(V)")
for (key, value) in sort(collect(d_min), by=last)
    println(rpad(key, 26, "."), lpad(round(value; digits=5), 6, "."))
end

println("\n\n\nMedian of time of computing sum(V)")
for (key, value) in sort(collect(d_med), by=last)
    println(rpad(key, 26, "."), lpad(round(value; digits=5), 6, "."))
end

# Don't work, still

In [ ]:
function ksintegrate(u₀, Lₓ, Δt, Nsteps, nsave);
    uⁿ = (1+0im)*u₀                      # force u to be complex
    Nₓ = length(uⁿ)                       # number of gridpoints
    kₓ = vcat(0:Nₓ/2 - 1, 0:0, -Nₓ/2+1:-1)  # integer wavenumbers: exp(2πikₓ x/L)
    α  = 2*pi*kₓ/Lₓ                       # real wavenumbers:    exp(iαx)
    D  = 1im*α                           # spectral D = d/dx operator 
    L  = α.^2 - α.^4                     # spectral L = -D^2 - D^4 operator
    G  = -0.5*D                          # spectral -1/2 D operator
    
    Nsave = div(Nsteps, nsave)+1         # number of saved u's, including t=0
    t = (0:Nsave-1)*(Δt*nsave)           # times of saved u's
    U = zeros(Nsave, Nₓ)                 # matrix of u(tᵢ, xⱼ) values
    U[1,:] = real(u₀)                    # assign initial condition to U
    s = 2                                # counter for saved data
    
    # construct diagonal matrices for LHS and RHS linear operators
    I   = ones(Nₓ)
    A⁻¹ = (I - Δt/2*L).^(-1)             
    B   = I + Δt/2*L
    
    # compute in-place FFTW plans
    FFT!  = plan_fft!(uⁿ,  flags=FFTW.ESTIMATE)
    IFFT! = plan_ifft!(uⁿ, flags=FFTW.ESTIMATE)

    # initialize nonlinear terms 
    Nuⁿ  = G.*fft(uⁿ.^2)      # set Nuⁿ = -1/2 d/dx (uⁿ)² = -u uₓ 
    Nuⁿ⁻¹ = copy(Nuⁿ)         # set Nuⁿ⁻¹ = Nuⁿ for first time step
    FFT!*uⁿ
    
    # timestepping loop
    for n = 0:Nsteps-1
        
        Nuⁿ⁻¹ .= Nuⁿ        # shift nonlinear term in time
        Nuⁿ  .= uⁿ         # store uⁿ in Nuⁿ in prep for collocation computation
        
        IFFT!*Nuⁿ          # transform to gridpt values of u
        Nuⁿ .= Nuⁿ.*Nuⁿ    # collocation calculation of u²
        FFT!*Nuⁿ           # transform u² back to spectral coeffs

        Nuⁿ .= G .* Nuⁿ      # compute  Nuⁿ = -1/2 d/dx u² = -u uₓ

        # Loop fusion! Julia unrolls this line into a single for-loop over vector elems,
        # avoiding the generation and allocation of temporary vectors you'd get in Matlab
        uⁿ .= A⁻¹ .* (B .* uⁿ .+ 3Δt/2 .* Nuⁿ .- Δt/2 .* Nuⁿ⁻¹)
        
        if mod(n, nsave) == 0
            U[s,:] = real(ifft(uⁿ))  # save into return matrix U 
            s += 1            
        end
    end
   
    t,U
end

In [ ]:

# set discretization parameters
Lx = 64π
Nx = 1024
Δt = 1/16
nsave = 8
Nsteps = 3200

# set initial condition
x = Lx*(0:Nx-1)/Nx
u₀ = cos.(x) + 0.1*sin.(x/8) + 0.002*cos.(x/32);

# compile the integrator function by running it one time step
t,U = ksintegrate(u₀, Lx, Δt, 8, nsave)

# integrate the PDE
t,U = ksintegrate(u₀, Lx, Δt, Nsteps, nsave) 

# plot results
heatmap(x,t, U, xlim=(x[1], x[end]), ylim=(t[1], t[end]), 
    xlabel="x", ylabel="t", title="u(x,t)", fillcolor=:bluesreds
)

In [ ]:
# set discretization parameters
Lx = 64π
Nx = 1024
Δt = 1/16
nsave = 8
Nsteps = 3200

# set initial condition
x = Lx*(0:Nx-1)/Nx
u₀ = cos.(x) + 0.1*sin.(x/8) + 0.002*cos.(x/32);

# compile the integrator function by running it one time step
t,U = ksintegrate(u₀, Lx, Δt, 8, nsave)

# integrate the PDE
t,U = ksintegrate(u₀, Lx, Δt, Nsteps, nsave) 

# plot results
heatmap(x,t, U, xlim=(x[1], x[end]), ylim=(t[1], t[end]), 
    xlabel="x", ylabel="t", title="u(x,t)", fillcolor=:bluesreds
)

# 6. User defined types
Exampled oryginaly by Andreas Noack [*Fast and Flexible Linear Algebra in Julia*](https://www.youtube.com/watch?v=VS0fnUOAKpI).

Consider field $\mathbb{Z} / p$ where $p$ is prime number. It is called Galois field $\textrm{GF}( p )$, we now implemented it in Julia.

In [ ]:
# Type definiton
struct GF{p, T} <: Number where {p, T<:Integer}
    rep::T  # Representation: intyger which holds the value of element in GF(p)
    
    function GF{p, T}(x::Integer) where {p, T<:Integer}
        return new(mod(x, p))
    end
end
GF{p}(x::T) where {p, T<:Integer} = GF{p, T}(x)

We now define some basic methods for $\textrm{GF}( p )$ that all Julia Numbers must have

In [ ]:
import Base: convert, inv, one, promote_rule, show, zero

function call(::Type{GF{p}}, x::Integer) where p
    if !isprime(p)
        throw(ArgumentError("p must be prime"))
    end
    return GF{p, typeof(x)}(mod(x, p))
end

convert(::Type{GF{p, T}}, x::Integer) where {p, T} = GF{p}(x)
convert(::Type{GF{p}}, x::Integer) where p = GF{p}(x)
convert(::Type{GF{p, T}}, x::GF{p}) where {p, T} = GF{p, T}(x.rep)
promote_rule(::Type{GF{p, T1}}, ::Type{T2}) where {p, T1, T2<:Integer} = GF{p, promote_type(T1, T2)}

# To be able show and print GF(p)
show(io::IO, x::GF) = show(io, x.rep)

Defining arithmetic operations on $\textrm{GF}( p )$

In [ ]:
GF{7}(9)

In [ ]:
GF{2}(4)

In [ ]:
typeof(GF{7}(0))

In [ ]:
x = GF{3}(4)

In [ ]:
x

In [ ]:
typeof(x)

In [ ]:
typeof(x.rep)

In [ ]:
GF{7}(2) + GF{7}(5)

In [ ]:
import Base: +

In [ ]:
+(x::GF{p, T}, y::GF{p, T}) where {p, T} = GF{p, T}(x.rep + y.rep)

In [ ]:
GF{3}(4) + GF{3}(2)

We now use some eldrih magic

In [ ]:
import Base: -, *, isless

for op in (:+, :-, :*, :isless)  # Loops over ops, defining each in terms of integer ops mod p
    @eval begin
        ($op)(x::GF{p, T}, y::GF{p, T}) where {p, T} = GF{p, T}($(op)(x.rep, y.rep))
    end
end

We now definde inverse and divison $\div$. This requariers more work, because dividing by zero throws error

In [ ]:
import Base: /

function inv(x::GF{p, T}) where {p, T}
    if x == zero(x)
        throw(DivideError())
    end
    r, u, v = gcdx(x.rep, p)
    GF{p, T}(u)
end
(/)(x::GF{p}, y::GF{p}) where p = x * inv(y);
abs(x::GF{p, T}) where {p, T} = x

In [ ]:
subtypes(Number)

In [ ]:
subtypes(Real)

In [ ]:
subtypes(AbstractFloat)

Some $\textrm{GF}( 7 )$ arithmetics

In [ ]:
x = GF{7}(9)   # x =  (9 mod 7) = 2
y = GF{7}(12)  # y = (12 mod 7) = 5

In [ ]:
@show x
@show y
@show x + y  # (2 + 5) mod 7 = 0
@show x - y  # (2 - 5) mod 7 = 4
@show x * y  # (2 * 5) mod 7 = 3
@show x / y  # (2 ÷ 5) mod 7 = 6, because 2 = (6 * 5) mod 7

In [ ]:
x = [GF{7}(rand(0:6)) for i = 1:3]

In [ ]:
y = [GF{7}(rand(0:6)) for i = 1:3]

In [ ]:
x + y

In [ ]:
x = [GF{7}(1) GF{7}(2) GF{7}(1)]

In [ ]:
y = [GF{7}(3), GF{7}(2), GF{7}(2)]

In [ ]:
y'

In [ ]:
Base.conj(x::GF{p,T}) where {p,T} = x

In [ ]:
y'